In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def cost_inv(file_path_scen,file_path_ref, run_name_scen,run_name_ref, output_folder):

    cost_inv = pd.read_csv(file_path_ref + 'Cost_Inv_' + run_name_ref + '.csv', sep = ',')
    new_inv_ref = cost_inv[(cost_inv['7'].str.contains('INV')) & (cost_inv['2'].str.contains('H2prd'))]
    new_inv_ref['group'] = new_inv_ref['3'].astype(str).str.cat(new_inv_ref['4'].astype(str), sep='_')
    new_inv_ref = new_inv_ref.sort_values(by='3', ascending=True)
    new_inv_ref['new_investment'] = new_inv_ref.groupby('group')['8'].transform('sum')
    new_inv_ref['new_investment'] = new_inv_ref['new_investment']/(10**3)
    new_inv_ref = new_inv_ref[~new_inv_ref['group'].duplicated()]

    cost_inv = pd.read_csv(file_path_scen + 'Cost_Inv_' + run_name_scen + '.csv', sep = ',')
    new_inv_scen = cost_inv[(cost_inv['7'].str.contains('INV')) & (cost_inv['2'].str.contains('H2prd'))]
    new_inv_scen['group'] = new_inv_scen['3'].astype(str).str.cat(new_inv_scen['4'].astype(str), sep='_')
    new_inv_scen = new_inv_scen.sort_values(by='3', ascending=True)
    new_inv_scen['new_investment'] = new_inv_scen.groupby('group')['8'].transform('sum')
    new_inv_scen['new_investment'] = new_inv_scen['new_investment']/(10**3)
    new_inv_scen = new_inv_scen[~new_inv_scen['group'].duplicated()]

    pivot_df = new_inv_ref.pivot(index='3', columns='4', values='new_investment')
    column_sums = pivot_df.sum(skipna=True)
    sorted_columns = column_sums.sort_values(ascending=False)
    sorted_df1 = pivot_df.reindex(sorted_columns.index, axis=1).fillna(0)

    pivot_df = new_inv_scen.pivot(index='3', columns='4', values='new_investment')
    sorted_df2 = pivot_df.reindex(sorted_columns.index, axis=1).fillna(0)


    def plot_stacked_bars_with_differentiation(df1, df2):
            fig, ax = plt.subplots(figsize=(12, 8))
            
            n_years1 = len(df1.index)
            n_years2 = len(df2.index)

            diff_index = abs(n_years1 - n_years2)

            if n_years1 > n_years2:
                indices1 = np.arange(n_years1)
                indices2 = np.arange(diff_index,n_years2+diff_index)
                plt_indices = indices1
                years = df1.index
            else:
                indices1 = np.arange(diff_index,n_years1+diff_index)
                indices2 = np.arange(n_years2)
                plt_indices = indices2
                years = df2.index

            bar_width = 0.35
            
            values1 = np.linspace(0, 1, 20)
            values2 = np.linspace(0, 1, 9)

            cmap1 = plt.cm.tab20
            cmap2 = plt.cm.Set3


            colors1 = cmap1(values1)
            colors2 = cmap2(values2)

            combined_colors = np.vstack((colors1, colors2))

            bottom_df1 = np.zeros(n_years1)
            for i, country in enumerate(df1.columns):
                ax.bar(indices1 - bar_width/2, df1[country], bar_width, bottom=bottom_df1, color=combined_colors[i],edgecolor='black', label=country)
                bottom_df1 += df1[country]
            
            bottom_df2 = np.zeros(n_years2)
            for i, country in enumerate(df2.columns):
                ax.bar(indices2 + bar_width/2, df2[country], bar_width, bottom=bottom_df2, color=combined_colors[i], hatch='/', edgecolor='black')
                bottom_df2 += df2[country]
            
            ax.set_xlabel('Year', fontsize=22) 
            ax.set_ylabel('$ Trillion', fontsize=22)
            ax.set_title('Investment costs', fontsize=22)
            ax.set_xticks(plt_indices)
            ax.set_xticklabels(years, fontsize=18) 
            first_legend = ax.legend(fontsize=14, fancybox=True, shadow=True, ncol=2,loc='upper left')
            ax.add_artist(first_legend) 

            empty_patch = Patch(facecolor='white', edgecolor='black', label='Reference')
            hashed_patch = Patch(facecolor='white', edgecolor='black', hatch='//', label='Scenario')
            second_legend = ax.legend(handles=[empty_patch, hashed_patch], labels=['Reference', 'Scenario'],
                                    fontsize=14, fancybox=True, shadow=True, bbox_to_anchor = (0.7, 1))
            ax.add_artist(second_legend)
            ax.grid(True)
            ax.set_axisbelow(True)
            plt.savefig(output_folder + 'H2investment_costs.pdf'
                    , format ='pdf'
                    ,  bbox_inches='tight')
    # # plt.show()
    plot_stacked_bars_with_differentiation(sorted_df1, sorted_df2)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 57)